In [226]:
import pandas as pd

In [227]:
df_charger =            pd.read_csv('./02_interim_data/ev_charger_monthly_interpolated.csv', encoding='utf-8-sig')
df_range_efficiency =   pd.read_csv('./02_interim_data/monthly_long_by_category.csv', encoding='utf-8-sig')
df_ev =                 pd.read_csv('./02_interim_data/vechicle_ev_data.csv', encoding='utf-8-sig')
df_total =              pd.read_csv('./02_interim_data/vechicle_total_data.csv', encoding='utf-8-sig')
df_hydrogen =           pd.read_csv('./02_interim_data/vechicle_hydrogen_data.csv', encoding='utf-8-sig')
df_hyb =                pd.read_csv('./02_interim_data/vechicle_hyb_data.csv', encoding='utf-8-sig')
df_temperature_rain =   pd.read_csv('./02_interim_data/temperature_rain_data.csv', encoding='utf-8-sig')
df_dust =               pd.read_csv('./02_interim_data/dust_data.csv', encoding='utf-8-sig') 
df_unemployment =       pd.read_csv('./01_raw_data/행정구역_시도__연령별_실업률_20250806081739.csv', encoding='cp949')
df_rate =               pd.read_csv('./01_raw_data/EV_project/경제 지표/기준금리.csv', encoding='cp949')
df_cpi =                pd.read_csv('./01_raw_data/지출목적별_소비자물가지수_품목포함__2020100__20250806082114.csv', encoding='cp949') 
df_income_total =       pd.read_csv('./01_raw_data/EV_project/경제 지표/지역별_자산__부채__소득_현황_20250806100101.csv', encoding='cp949')
df_subsidy =            pd.read_csv('./01_raw_data/전기차 보조금 최종.csv', encoding='cp949')

In [228]:
df = df_ev.copy()
df = df.drop(columns=['연료'])
df.rename(columns={'등록수': '전기차 등록수','순증가량':'전기차 순증가량'}, inplace=True)

In [229]:

df_range_efficiency = df_range_efficiency.rename(columns={'년': '년도'})

df[['년도', '월']]  = df[['년도', '월']].astype(int)
df_range_efficiency[['년도', '월']] = df_range_efficiency[['년도', '월']].astype(int)
df['차종'] = df['차종'].astype(str)
df_range_efficiency['차종'] = df_range_efficiency['차종'].astype(str)

df_merged = pd.merge(
    df,
    df_range_efficiency,
    on=['년도', '월', '차종'],
    how='left'
)

def fill_special_cases(group):
    # “승합” ← “승용”
    if '승용' in group['차종'].values:
        vals = group.loc[group['차종']=='승용', ['주행거리','에너지소비효율','충전지용량']].iloc[0]
        group.loc[group['차종']=='승합', ['주행거리','에너지소비효율','충전지용량']] = vals.values
    # “특수” ← “화물”
    if '화물' in group['차종'].values:
        vals = group.loc[group['차종']=='화물', ['주행거리','에너지소비효율','충전지용량']].iloc[0]
        group.loc[group['차종']=='특수', ['주행거리','에너지소비효율','충전지용량']] = vals.values
    return group

df_merged = (
    df_merged
    .groupby(['년도','월'], group_keys=False)
    .apply(fill_special_cases)
)

# 5) 결과 확인

# 6) (선택) CSV 저장
df_merged.to_csv('merged_filled_dataset.csv', index=False, encoding='utf-8-sig')
df_merged



C:\Users\junjo\AppData\Local\Temp\ipykernel_134612\2786394572.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_special_cases)


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889
...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667


In [230]:
df_charger

,연월,시도,급속 설치건수,급속 누적건수,완속 설치건수,완속 누적건수
0,2014-01,강원,2,2,1,1
1,2014-02,강원,0,2,0,1
2,2014-03,강원,0,2,0,1
3,2014-04,강원,0,2,0,1
4,2014-05,강원,0,2,0,1
...,...,...,...,...,...,...
3235,2028-08,충북,0,2220,0,12939
3236,2028-09,충북,0,2220,0,12939
3237,2028-10,충북,0,2220,0,12939
3238,2028-11,충북,0,2220,0,12939


In [231]:
import pandas as pd

# 1) 연월에서 년·월 분리
df_charger[['년도','월']] = (
    df_charger['연월']
    .str.split('-', expand=True)
    .rename(columns={0:'년도', 1:'월'})
    .astype(int)
)

# 2) 컬럼명 통일: 시도 → 지역
df_charger = df_charger.rename(columns={'시도': '지역'})

# 3) 병합에 필요한 컬럼만 선택
cols_to_merge = [
    '년도','월','지역',
    '급속 설치건수','급속 누적건수',
    '완속 설치건수','완속 누적건수'
]
df_charger_sel = df_charger[cols_to_merge]

# 4) df_merged와 병합
df_final = pd.merge(
    df_merged,       # 기존에 “등록+성능” 데이터가 담긴 df_merged
    df_charger_sel,
    on=['년도','월','지역'],
    how='left'       # df_merged 기준으로 설치건수만 붙임
)
df_final = df_final.rename(columns={
    '급속 설치건수':'급속 충전기 설치건수',
    '급속 누적건수':'급속 충전기 누적건수',
    '완속 설치건수':'완속 충전기 설치건수',
    '완속 누적건수':'완속 충전기 누적건수'
})
# 5) 결과 확인
df_final
# (필요 시) df_final.fillna(0) 등으로 NaN 처리
# df_final.to_csv('merged_all.csv', index=False, encoding='utf-8-sig')


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,급속 충전기 설치건수,급속 충전기 누적건수,완속 충전기 설치건수,완속 충전기 누적건수
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,12,684,36,973
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,12,684,36,973
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,12,684,36,973
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,12,684,36,973
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,12,684,36,973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,7,2213,167,12772
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,7,2213,167,12772
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,7,2213,167,12772
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,7,2213,167,12772


In [232]:
import pandas as pd

# 1) df_total 준비: "연료=총계" 전체 집계 데이터
# 1-1) 자료형 통일
df_total[['년도','월']] = df_total[['년도','월']].astype(int)
df_total['지역'] = df_total['지역'].astype(str).str.strip()
df_total['차종'] = df_total['차종'].astype(str).str.strip()
df_total['용도'] = df_total['용도'].astype(str).str.strip()

# 1-2) 충돌 방지용 컬럼명 변경
df_total = df_total.rename(
    columns={
        '등록수':     '등록수_총계',
        '순증가량':   '순증가량_총계'
    }
)

# 1-3) 병합에 쓸 컬럼만 선택
cols_tot = [
    '년도','월','지역','차종','용도',
    '등록수_총계','순증가량_총계'
]
df_total_sel = df_total[cols_tot]

# 2) df_final 과 병합
df_merged_tot = pd.merge(
    df_final,          # 기존에 전기+성능+충전소까지 붙여둔 df_final
    df_total_sel,      # 전체 연료 합산치
    on=['년도','월','지역','차종','용도'],
    how='left'         # df_final 기준, 일치하는 총계만 붙이기
)
df_merged_tot = df_merged_tot.rename(columns={
    '등록수_총계':   '차량 등록수_총계',
    '순증가량_총계': '차량 순증가량_총계'
})



# 3) 결과 확인
df_merged_tot

# # 4) (선택) NaN을 0으로
# df_merged_tot[['등록수_총계','순증가량_총계']] = (
#     df_merged_tot[['등록수_총계','순증가량_총계']]
#     .fillna(0)
# )


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,급속 충전기 설치건수,급속 충전기 누적건수,완속 충전기 설치건수,완속 충전기 누적건수,차량 등록수_총계,차량 순증가량_총계
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,12,684,36,973,784808,2108
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,12,684,36,973,759447,2084
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,12,684,36,973,25361,24
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,12,684,36,973,596080,1703
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,12,684,36,973,584095,1691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,7,2213,167,12772,3680,29
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,7,2213,167,12772,2932,13
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,7,2213,167,12772,166638,35
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,7,2213,167,12772,150399,43


In [233]:
import pandas as pd

# 공통 키
keys = ['년도','월','지역','차종','용도']

def prepare_and_merge(base_df, src_df, prefix):
    """
    base_df 에 src_df 의 등록수/순증가량(원본 컬럼명 '등록수','순증가량')을
    prefix를 붙인 새 컬럼으로 합친 후 반환.
    """
    # 1) 타입·문자열 정리
    src = src_df.copy()
    src[['년도','월']] = src[['년도','월']].astype(int)
    for c in ['지역','차종','용도']:
        src[c] = src[c].astype(str).str.strip()
    
    # 2) 컬럼명 변경
    src = src.rename(columns={
        '등록수':   f'{prefix} 등록수',
        '순증가량': f'{prefix} 순증가량'
    })
    
    # 3) 병합용 컬럼만 선택
    cols = keys + [f'{prefix} 등록수', f'{prefix} 순증가량']
    src_sel = src[cols]
    
    # 4) 병합
    merged = pd.merge(
        base_df,
        src_sel,
        on=keys,
        how='left'
    )
    return merged

# 0) 시작: 기존에 df_merged_tot 에 df_total(전체연료)까지 붙여둔 상태라고 가정
df_full = df_merged_tot

# 1) 수소차 데이터 병합
df_full = prepare_and_merge(df_full, df_hydrogen, prefix='수소차')

# 2) 하이브리드차 데이터 병합
df_full = prepare_and_merge(df_full, df_hyb, prefix='하이브리드차')

# 3) 결과 확인
df_full

,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,급속 충전기 설치건수,급속 충전기 누적건수,완속 충전기 설치건수,완속 충전기 누적건수,차량 등록수_총계,차량 순증가량_총계,수소차 등록수,수소차 순증가량,하이브리드차 등록수,하이브리드차 순증가량
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,12,684,36,973,784808,2108,784808,2108,14145,229
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,12,684,36,973,759447,2084,759447,2084,14098,225
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,12,684,36,973,25361,24,25361,24,47,4
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,12,684,36,973,596080,1703,596080,1703,14145,229
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,12,684,36,973,584095,1691,584095,1691,14098,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,7,2213,167,12772,3680,29,3680,29,2,1
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,7,2213,167,12772,2932,13,2932,13,0,0
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,7,2213,167,12772,166638,35,166638,35,84,5
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,7,2213,167,12772,150399,43,150399,43,83,4


In [234]:
import pandas as pd

# 1) df_temperature_rain 처리: “일시” → “년도”, “월” 분리, “시도” → “지역”
df_temp = df_temperature_rain.copy()
df_temp[['년도','월']] = (
    df_temp['일시']
    .str.split('-', expand=True)
    .astype(int)
)
df_temp = df_temp.rename(columns={'시도': '지역'})
df_temp['지역'] = df_temp['지역'].astype(str).str.strip()

# 2) 병합에 쓸 컬럼만 선택
cols_temp = [
    '년도','월','지역',
    '평균기온(°C)','평균최고기온(°C)',
    '평균최저기온(°C)','월합강수량(00~24h만)(mm)'
]
df_temp_sel = df_temp[cols_temp]

# 3) df_full(또는 df_merged_tot 기준) 과 병합
df_full = pd.merge(
    df_full,      # 기존에 전기·성능·충전·총계·수소·하이브리드까지 합친 데이터
    df_temp_sel,
    on=['년도','월','지역'],
    how='left'    # df_full 기준, 일치하는 기상데이터만 붙이기
)
df_full.columns = df_full.columns.str.replace(r'_(?:x|y)$', '', regex=True)
# 4) 결과 확인
df_full

# (선택) 필요 시 NaN을 채우려면:
# df_full[['평균기온(°C)','평균최고기온(°C)','평균최저기온(°C)','월합강수량(00~24h만)(mm)']] = \
#     df_full[['평균기온(°C)','평균최고기온(°C)','평균최저기온(°C)','월합강수량(00~24h만)(mm)']].fillna(0)


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,차량 등록수_총계,차량 순증가량_총계,수소차 등록수,수소차 순증가량,하이브리드차 등록수,하이브리드차 순증가량,평균기온(°C),평균최고기온(°C),평균최저기온(°C),월합강수량(00~24h만)(mm)
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,784808,2108,784808,2108,14145,229,0.728571,5.821429,-3.328571,85.335714
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,759447,2084,759447,2084,14098,225,0.728571,5.821429,-3.328571,85.335714
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,25361,24,25361,24,47,4,0.728571,5.821429,-3.328571,85.335714
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,596080,1703,596080,1703,14145,229,0.728571,5.821429,-3.328571,85.335714
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,584095,1691,584095,1691,14098,225,0.728571,5.821429,-3.328571,85.335714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,3680,29,3680,29,2,1,0.040000,4.900000,-4.600000,6.040000
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,2932,13,2932,13,0,0,0.040000,4.900000,-4.600000,6.040000
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,166638,35,166638,35,84,5,0.040000,4.900000,-4.600000,6.040000
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,150399,43,150399,43,83,4,0.040000,4.900000,-4.600000,6.040000


In [235]:
df_dust = df_dust.rename(columns={'년': '년도'})

# 2) 자료형·문자열 정리
df_dust[['년도','월']] = df_dust[['년도','월']].astype(int)
df_dust['지역']         = df_dust['지역'].astype(str).str.strip()

# 3) 병합에 필요한 컬럼만 추출
dust_cols = ['년도','월','지역','미세먼지2.5 수치','미세먼지10 수치']
df_dust_sel = df_dust[dust_cols]

# 4) df_full과 병합
df_merged_all = pd.merge(
    df_full,      # 기존 전기·성능·충전·총계·수소·하이브·기상까지 합친 데이터
    df_dust_sel,  # 미세먼지
    on=['년도','월','지역'],
    how='left'    # df_full 기준으로, 미세먼지 있으면 붙이고 없으면 NaN
)

# 5) (선택) NaN을 0 혹은 다른 값으로 채우기
df_merged_all[['미세먼지2.5 수치','미세먼지10 수치']] = (
    df_merged_all[['미세먼지2.5 수치','미세먼지10 수치']]
    .fillna(0)
)

# 6) 결과 확인
df_merged_all

,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,수소차 등록수,수소차 순증가량,하이브리드차 등록수,하이브리드차 순증가량,평균기온(°C),평균최고기온(°C),평균최저기온(°C),월합강수량(00~24h만)(mm),미세먼지2.5 수치,미세먼지10 수치
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,784808,2108,14145,229,0.728571,5.821429,-3.328571,85.335714,24.0,34.0
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,759447,2084,14098,225,0.728571,5.821429,-3.328571,85.335714,24.0,34.0
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,25361,24,47,4,0.728571,5.821429,-3.328571,85.335714,24.0,34.0
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,596080,1703,14145,229,0.728571,5.821429,-3.328571,85.335714,24.0,34.0
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,584095,1691,14098,225,0.728571,5.821429,-3.328571,85.335714,24.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,3680,29,2,1,0.040000,4.900000,-4.600000,6.040000,20.0,33.0
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,2932,13,0,0,0.040000,4.900000,-4.600000,6.040000,20.0,33.0
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,166638,35,84,5,0.040000,4.900000,-4.600000,6.040000,20.0,33.0
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,150399,43,83,4,0.040000,4.900000,-4.600000,6.040000,20.0,33.0


In [236]:

tmp = df_cpi['시점'].astype(str).str.split('.', expand=True)
df_cpi['년도'] = tmp[0].astype(int)
# “1”만 있는 것은 Excel이 10월을 1로 잘못 읽은 것 → 10으로 복원
df_cpi['월'] = tmp[1].apply(lambda m: 10 if m == '1' else int(m))

df_cpi = df_cpi.rename(columns={
    '시도별': '지역',
    '데이터': 'cpi'
})

# 1-3) 지역명 약어 처리 (예: “서울특별시” → “서울”, “경기도” → “경기” 등)
region_map = {
    '전국': '전국',
    '서울특별시': '서울',
    '부산광역시': '부산',
    '대구광역시': '대구',
    '인천광역시': '인천',
    '광주광역시': '광주',
    '대전광역시': '대전',
    '울산광역시': '울산',
    '세종특별자치시': '세종',
    '경기도': '경기',
    '강원특별자치도': '강원',
    '충청북도': '충북',
    '충청남도': '충남',
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '제주특별자치도': '제주'
}
df_cpi['지역'] = df_cpi['지역'].replace(region_map)

# 2) 병합용 컬럼만 선택
df_cpi_sel = df_cpi[['년도','월','지역','cpi']]

# 3) df_full 과 병합
df_merged_all = pd.merge(
    df_merged_all,
    df_cpi_sel,
    on=['년도','월','지역'],
    how='left'    # df_full 기준으로, 일치하는 CPI 값만 붙이기
)
df_merged_all.columns = df_merged_all.columns.str.replace(r'_(?:x|y)$', '', regex=True)
# 4) 결과 확
df_merged_all

,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,수소차 순증가량,하이브리드차 등록수,하이브리드차 순증가량,평균기온(°C),평균최고기온(°C),평균최저기온(°C),월합강수량(00~24h만)(mm),미세먼지2.5 수치,미세먼지10 수치,cpi
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,2108,14145,229,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,2084,14098,225,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,24,47,4,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,1703,14145,229,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,1691,14098,225,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,29,2,1,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,13,0,0,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,35,84,5,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,43,83,4,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50


In [237]:
df_unemployment

,시도별,시점,연령계층별,데이터
0,계,2020.1/4,계,4.2
1,계,2020.2/4,계,4.4
2,계,2020.3/4,계,3.6
3,계,2020.4/4,계,3.7
4,계,2021.1/4,계,5.0
...,...,...,...,...
355,제주도,2023.4/4,계,1.4
356,제주도,2024.1/4,계,2.6
357,제주도,2024.2/4,계,2.3
358,제주도,2024.3/4,계,1.9


In [238]:
import pandas as pd

# 1) df_unemployment 준비
#    가정: df_unemployment에는 다음 컬럼이 있습니다.
#      - '시도별'        (예: '서울특별시', '경기도' 등)
#      - '시점'          (예: '2020.01' 또는 '2020-01' 형태)
#      - '실업률' 혹은 '데이터' (월별 실업률 값)

# 1-1) ‘시점’을 ‘년도’, ‘월’로 분리하기
# 문자열이 아닐 경우 astype(str) 먼저 해주세요
# 1-1) “시점”에서 년도·분기 분리하고, 분기를 월로 확장
tmp = df_unemployment['시점'].astype(str).str.split('.', expand=True)
df_unemployment['년도'] = tmp[0].astype(int)
df_unemployment['분기'] = tmp[1].str.split('/', expand=True)[0].astype(int)

# 분기→월 리스트 매핑
months_map = {1:[1,2,3], 2:[4,5,6], 3:[7,8,9], 4:[10,11,12]}
df_unemployment['월'] = df_unemployment['분기'].map(months_map)

# explode 해서 월별로 늘리기
df_unemployment = df_unemployment.explode('월').reset_index(drop=True)

# (선택) ‘분기’ 칼럼이 더 필요 없으면 제거
df_unemployment = df_unemployment.drop(columns='분기')
# 1-2) 컬럼명 통일
# 컬럼명이 '데이터'인 경우 → '실업률'로, 
# 이미 '실업률'이면 생략하셔도 됩니다
df_unemployment = df_unemployment.rename(columns={
    '시도별': '지역',
    '데이터': '실업률'
})

# 1-3) 지역명 약어 매핑 (풀네임 → 약어)
region_map = {
    '전국': '전국','서울특별시': '서울','부산광역시': '부산',
    '대구광역시': '대구','인천광역시': '인천','광주광역시': '광주',
    '대전광역시': '대전','울산광역시': '울산','세종특별자치시': '세종',
    '경기도': '경기','강원특별자치도': '강원','충청북도': '충북','충청남도': '충남',
    '전라북도': '전북','전라남도': '전남','경상북도': '경북','경상남도': '경남',
    '제주도': '제주','계':'전국','강원도': '강원'
}
df_unemployment['지역'] = df_unemployment['지역'].replace(region_map)

# 1-4) 병합용 컬럼만 선택
unemp_cols = ['년도','월','지역','실업률']
df_unemp_sel = df_unemployment[unemp_cols]

# 2) df_merged_all에 병합
df_merged_all = pd.merge(
    df_merged_all,
    df_unemp_sel,
    on=['년도','월','지역'],
    how='left'         # 기존 월별 종합 데이터 기준으로 실업률 붙이기
)
df_merged_all.columns = df_merged_all.columns.str.replace(r'_(?:x|y)$', '', regex=True)
# 3) 결과 확인
df_merged_all


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,하이브리드차 등록수,하이브리드차 순증가량,평균기온(°C),평균최고기온(°C),평균최저기온(°C),월합강수량(00~24h만)(mm),미세먼지2.5 수치,미세먼지10 수치,cpi,실업률
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,14145,229,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39,5.1
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,14098,225,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39,5.1
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,47,4,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39,5.1
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,14145,229,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39,5.1
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,14098,225,0.728571,5.821429,-3.328571,85.335714,24.0,34.0,100.39,5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,2,1,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50,2.3
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,0,0,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50,2.3
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,84,5,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50,2.3
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,83,4,0.040000,4.900000,-4.600000,6.040000,20.0,33.0,115.50,2.3


In [239]:
df_rate

,변경일자,Unnamed: 1,기준금리
0,2025,05월 29일,2.50
1,2025,02월 25일,2.75
2,2024,11월 28일,3.00
3,2024,10월 11일,3.25
4,2023,01월 13일,3.50
5,2022,11월 24일,3.25
6,2022,10월 12일,3.00
7,2022,08월 25일,2.50
8,2022,07월 13일,2.25
9,2022,05월 26일,1.75


In [240]:
# 1) '변경일자' + 'Unnamed: 1' 합쳐서 datetime으로 파싱


df_rate['date'] = pd.to_datetime(
    df_rate['변경일자'].astype(str) + ' ' + df_rate['Unnamed: 1'],
    format='%Y %m월 %d일'
)

# 2) 기준금리만 남기고 정렬
df_rate = df_rate[['date', '기준금리']].sort_values('date')

# 3) 일별로 확장 — 이전 값으로 채워넣기
daily_rate = (
    df_rate
    .set_index('date')
    .resample('D')
    .ffill()
    .reset_index()
)

# 4) 월별 평균 계산
monthly_rate = (
    daily_rate
    .groupby(daily_rate['date'].dt.to_period('M'))
    .agg({'기준금리': 'mean'})
    .reset_index()
)
monthly_rate['년도'] = monthly_rate['date'].dt.year
monthly_rate['월']   = monthly_rate['date'].dt.month
monthly_rate = monthly_rate[['년도', '월', '기준금리']] \
    .rename(columns={'기준금리': '평균기준금리'})

# 5) df_merged_all 과 병합
df_merged_all = pd.merge(
    df_merged_all,
    monthly_rate,
    on=['년도', '월'],
    how='left'
)

# 6) 결과 확인


In [241]:
df_income = df_income_total.copy()
df_income

,부채보유 여부별,지역별,시점,가구분포 (%),가구원수 (명),가구주 연령 (세),경상소득(전년도) (만원),자산 (만원),부채 (만원),순자산액 (만원)
0,전체,전국,2020,100.0,2.6,56.1,5966,44543,8256,36287
1,전체,전국,2021,100.0,2.6,55.6,6180,50253,8801,41452
2,전체,전국,2022,100.0,2.5,55.2,6470,54772,9170,45602
3,전체,전국,2023,100.0,2.4,55.0,6762,52727,9186,43540
4,전체,전국,2024,100.0,2.4,54.9,7185,54022,9128,44894
...,...,...,...,...,...,...,...,...,...,...
85,전체,제주특별자치도,2020,1.3,2.7,56.8,5571,52147,7869,44278
86,전체,제주특별자치도,2021,1.3,2.8,56.6,5964,56760,7764,48995
87,전체,제주특별자치도,2022,1.3,2.7,55.9,6289,57466,7227,50238
88,전체,제주특별자치도,2023,1.3,2.6,56.1,6460,51170,5776,45394


In [242]:
import pandas as pd


# 1) 컬럼명 정리
df_income = df_income.rename(columns={
    '지역별': '지역',
    '시점':   '년도'
})

# 2) 자료형 통일
df_income['년도'] = df_income['년도'].astype(int)
df_income['지역'] = df_income['지역'].astype(str).str.strip()

# 3) 지역명 풀네임 → 약어 매핑 (정규식 없이)
region_map = {
    '전국': '전국',
    '서울특별시': '서울',
    '부산광역시': '부산',
    '대구광역시': '대구',
    '인천광역시': '인천',
    '광주광역시': '광주',
    '대전광역시': '대전',
    '울산광역시': '울산',
    '세종특별자치시': '세종',
    '경기도': '경기',
    '강원특별자치도': '강원',
    '충청북도': '충북',
    '충청남도': '충남',
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '제주특별자치도': '제주'
}
df_income['지역'] = df_income['지역'].replace(region_map)

# 4) 필요한 컬럼만 선택 및 컬럼명 간소화
df_income_sel = df_income[[
    '년도', '지역',
    '경상소득(전년도) (만원)',
    '자산 (만원)',
    '부채 (만원)',
    '순자산액 (만원)'
]].rename(columns={
    '경상소득(전년도) (만원)': '경상소득',
    '자산 (만원)':            '자산',
    '부채 (만원)':            '부채',
    '순자산액 (만원)':        '순자산액'
})

# 5) df_merged_all에 병합
df_merged_all = pd.merge(
    df_merged_all,
    df_income_sel,
    on=['년도', '지역'],
    how='left'
)
df_merged_all.columns = df_merged_all.columns.str.replace(r'_(?:x|y)$', '', regex=True)
# 6) 결과 확인
df_merged_all


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,월합강수량(00~24h만)(mm),미세먼지2.5 수치,미세먼지10 수치,cpi,실업률,평균기준금리,경상소득,자산,부채,순자산액
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,85.335714,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,85.335714,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,85.335714,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,85.335714,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,85.335714,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,6.040000,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,6.040000,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,6.040000,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,6.040000,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619


In [ ]:
import pandas as pd

# 0) 원본 df_subsidy: ['연도','월','지역','차량유형','기준보조금','월별평균','조정보조금']

# 1) '조정보조금' → '보조금'으로 이름 변경
df_subsidy = df_subsidy.rename(columns={'조정보조금': '보조금'})

# 2) 병합 키 이름 통일
#    연도 → 년도, 차량유형 → 차종 (df_merged_all에 따라 조정)
df_subsidy = df_subsidy.rename(columns={
    '연도':     '년도',
    '차량유형': '차종'
})

# 3) 키 컬럼 타입 정리
df_subsidy[['년도','월']] = df_subsidy[['년도','월']].astype(int)
df_subsidy['지역'] = df_subsidy['지역'].astype(str).str.strip()
df_subsidy['차종'] = df_subsidy['차종'].astype(str).str.strip()

# 4) 병합에 쓸 컬럼만 선택
sub_cols = ['년도','월','지역','차종','보조금']
df_subsidy_sel = df_subsidy[sub_cols]

# 5) df_merged_all과 병합
df_merged_all = pd.merge(
    df_merged_all,
    df_subsidy_sel,
    on=['년도','월','지역','차종'],
    how='left'
)



     년도  월  지역  차종         보조금
0  2020  1  강원  소계         NaN
1  2020  1  강원  소계         NaN
2  2020  1  강원  소계         NaN
3  2020  1  강원  승용  559.784946
4  2020  1  강원  승용  559.784946


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,미세먼지2.5 수치,미세먼지10 수치,cpi,실업률,평균기준금리,경상소득,자산,부채,순자산액,보조금
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,NaN
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,NaN
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,NaN
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,559.784946
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,559.784946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,NaN
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,NaN
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,1448.606061
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,1448.606061


In [246]:
# 6) 결과 확인
print(df_merged_all[['년도','월','지역','차종','보조금']].head())
df_merged_all

     년도  월  지역  차종         보조금
0  2020  1  강원  소계         NaN
1  2020  1  강원  소계         NaN
2  2020  1  강원  소계         NaN
3  2020  1  강원  승용  559.784946
4  2020  1  강원  승용  559.784946


,년도,월,지역,차종,용도,전기차 등록수,전기차 순증가량,주행거리,에너지소비효율,충전지용량,...,미세먼지2.5 수치,미세먼지10 수치,cpi,실업률,평균기준금리,경상소득,자산,부채,순자산액,보조금
0,2020,1,강원,소계,계,2448,3,359.976190,4.621429,221.500000,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,NaN
1,2020,1,강원,소계,비사업용,2303,3,359.976190,4.621429,221.500000,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,NaN
2,2020,1,강원,소계,사업용,145,0,359.976190,4.621429,221.500000,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,NaN
3,2020,1,강원,승용,계,2426,3,399.361111,4.741667,227.888889,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,559.784946
4,2020,1,강원,승용,비사업용,2300,3,399.361111,4.741667,227.888889,...,24.0,34.0,100.39,5.1,1.25,4962,34307,5619,28689,559.784946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,2024,12,충북,특수,비사업용,16,2,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,NaN
16196,2024,12,충북,특수,사업용,0,0,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,NaN
16197,2024,12,충북,화물,계,7219,73,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,1448.606061
16198,2024,12,충북,화물,비사업용,6253,55,212.846154,2.900000,163.316667,...,20.0,33.0,115.50,2.3,3.00,6939,41100,6481,34619,1448.606061


In [247]:
# 1) 소계·승합·특수 행을 별도 DataFrame으로 분리
df_summary = df_merged_all[
    df_merged_all['차종'].isin(['소계', '승합', '특수'])
].copy()

# 2) 원본에서는 화물·승용만 남기기
df_merged_all = df_merged_all[
    df_merged_all['차종'].isin(['화물', '승용'])
].copy()

# 3) 인덱스 재설정(선택)
df_summary     = df_summary.reset_index(drop=True)
df_merged_all  = df_merged_all.reset_index(drop=True)

# 이제
# - df_summary 에는 소계·승합·특수 행만,
# - df_merged_all 에는 화물·승용 행만 남아 있습니다.


In [248]:
df_merged_all.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [245]:

# def add_input_vars(df: pd.DataFrame,
#                    conditions: dict,
#                    new_vars: dict,
#                    default_val=None) -> pd.DataFrame:
#     """
#     지정한 조건에 맞는 행들에 입력 변수를 추가/업데이트,
#     new_vars에 들어있는 컬럼이 df에 없으면 default_val로 컬럼 먼저 생성합니다.
#     """
#     df = df.copy()
    
#     # 1) new_vars에 있는 컬럼이 없으면 default_val로 채워서 미리 생성
#     for col in new_vars:
#         if col not in df.columns:
#             df[col] = default_val
    
#     # 2) 조건 마스크 생성
#     mask = pd.Series(True, index=df.index)
#     for col, val in conditions.items():
#         mask &= (df[col] == val)
    
#     # 3) 조건에 맞는 행들에 대해 값 할당
#     for new_col, new_val in new_vars.items():
#         df.loc[mask, new_col] = new_val
    
#     return df


# # # 사용 예시
# # conds = {
# #     '년도': 2020, '월': 1, '지역': '경기',
# #     '연료': '전기', '차종': '승용', '용도': '계'
# # }
# # updates = {
# #     'oil_price': 1300,
# #     'subsidy': 1500000,
# #     'charging_stations': 42   # 신규 컬럼도 바로 추가됩니다
# # }

# # df_full = add_input_vars(df_full, conds, updates, default_val=0)

# # # 확인
# # print(df_full.loc[
# #     (df_full['년도']==2020)&
# #     (df_full['월']==1)&
# #     (df_full['지역']=='경기')&
# #     (df_full['연료']=='전기')&
# #     (df_full['차종']=='승용')&
# #     (df_full['용도']=='계')
# # ])